In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
# import numpy as np

In [3]:
database_type = "/training"

#### Reading in Training Tables


In [4]:
lab = pd.read_csv('../eICU' + database_type + '/lab.csv')
lab

,patientunitstayid,labresultoffset,creatinine,wbcx1000,lactate
0,1754323,-647,0.90,5.4,NaN
1,1754323,603,0.94,12.8,NaN
2,1754323,3579,0.74,NaN,NaN
3,1754323,2113,0.87,9.5,NaN
4,1754323,5073,0.87,8.4,NaN
...,...,...,...,...,...
14764,2715137,1951,0.60,NaN,NaN
14765,2715137,-29,0.60,NaN,NaN
14766,2715137,1001,0.60,NaN,NaN
14767,2754778,1282,1.05,6.7,NaN


In [5]:
vitalperiodic = pd.read_csv('../eICU' + database_type + '/vitalPeriodic.csv')
vitalperiodic

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic
0,141765,1179,NaN,82.0,NaN,NaN
1,141765,189,NaN,76.0,30.0,NaN
2,141765,1169,NaN,84.0,NaN,NaN
3,141765,1534,NaN,92.0,NaN,NaN
4,141765,1164,NaN,86.0,NaN,NaN
...,...,...,...,...,...,...
1634955,3353113,1846,NaN,67.0,19.0,NaN
1634956,3353113,1146,NaN,65.0,18.0,NaN
1634957,3353113,366,NaN,48.0,16.0,NaN
1634958,3353113,1841,NaN,67.0,18.0,NaN


In [6]:
# The original dataframes were deleted to free up resources.  
merge = pd.merge(vitalperiodic, lab,  how='outer', left_on=['patientunitstayid','observationoffset'], right_on = ['patientunitstayid','labresultoffset'])
del lab, vitalperiodic
merge 

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,labresultoffset,creatinine,wbcx1000,lactate
0,141765,1179.0,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN
1,141765,189.0,NaN,76.0,30.0,NaN,NaN,NaN,NaN,NaN
2,141765,1169.0,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN
3,141765,1534.0,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN
4,141765,1164.0,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1648290,2715137,NaN,NaN,NaN,NaN,NaN,1951.0,0.60,NaN,NaN
1648291,2715137,NaN,NaN,NaN,NaN,NaN,-29.0,0.60,NaN,NaN
1648292,2715137,NaN,NaN,NaN,NaN,NaN,1001.0,0.60,NaN,NaN
1648293,2754778,NaN,NaN,NaN,NaN,NaN,1282.0,1.05,6.7,NaN


In [7]:
intakeOutputUrine = pd.read_csv('../eICU' + database_type + '/intakeOutputUrine.csv')
intakeOutputUrine

,patientunitstayid,intakeoutputoffset,urineoutputbyweight
0,141764,-14,3.003003
1,141765,1401,12.903226
2,141765,556,10.752688
3,141765,1936,10.752688
4,141765,2056,11.827957
...,...,...,...
33197,3353113,1869,2.556818
33198,3353113,1089,0.284091
33199,3353113,2169,1.136364
33200,3353113,2109,1.363636


In [8]:
merge1 = pd.merge(merge, intakeOutputUrine,  how='outer', left_on=['patientunitstayid','observationoffset'], right_on = ['patientunitstayid','intakeoutputoffset'])
del merge, intakeOutputUrine
merge1

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,labresultoffset,creatinine,wbcx1000,lactate,intakeoutputoffset,urineoutputbyweight
0,141765,1179.0,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141765,189.0,NaN,76.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141765,1169.0,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,141765,1534.0,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141765,1164.0,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1676934,3353113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1869.0,2.556818
1676935,3353113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1089.0,0.284091
1676936,3353113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2169.0,1.136364
1676937,3353113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2109.0,1.363636


In [9]:
merge1['observationoffset'] = merge1.apply(lambda x: x['observationoffset'] if pd.notna(x['observationoffset']) else (x['labresultoffset'] if pd.notna(x['labresultoffset']) else x['intakeoutputoffset']), axis = 1)
merge1 = merge1.drop(['labresultoffset', 'intakeoutputoffset'], axis = 1)
merge1

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight
0,141765,1179.0,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN
1,141765,189.0,NaN,76.0,30.0,NaN,NaN,NaN,NaN,NaN
2,141765,1169.0,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN
3,141765,1534.0,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN
4,141765,1164.0,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1676934,3353113,1869.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.556818
1676935,3353113,1089.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.284091
1676936,3353113,2169.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.136364
1676937,3353113,2109.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.363636


In [10]:
diagnosis = pd.read_csv('../eICU' + database_type + '/diagnosis.csv')
diagnosis = diagnosis.astype({'diagnosis': int})
diagnosis

,patientunitstayid,diagnosisoffset,diagnosis
0,346380,5028,0
1,346380,685,0
2,346380,5035,0
3,346380,5035,0
4,346380,5028,0
...,...,...,...
24973,3158501,11,0
24974,3158501,11,0
24975,3158501,11,0
24976,3158501,11,0


In [11]:
finalMerge = pd.merge(merge1, diagnosis,  how='outer', left_on=['patientunitstayid','observationoffset'], right_on = ['patientunitstayid', 'diagnosisoffset'])
del merge1, diagnosis
finalMerge.drop_duplicates()
finalMerge

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosisoffset,diagnosis
0,141765,1179.0,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141765,189.0,NaN,76.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141765,1169.0,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,141765,1534.0,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141765,1164.0,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1700915,3158501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,0.0
1700916,3158501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,0.0
1700917,3158501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,0.0
1700918,3158501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,0.0


In [12]:
path = "../eICU/training/"
finalMerge.to_csv(path + "finalMerge.csv", sep=',', index=False, encoding='utf-8')
